In [1]:
import pandas as pd
import numpy as np 
import re, glob, sys, os
import matplotlib.pyplot as plt 
from matplotlib.cm import get_cmap
import cartopy.io.img_tiles as cimgt
import cartopy.crs as ccrs
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from netCDF4 import Dataset as netcdf_dataset
from scipy.ndimage import gaussian_filter



In [120]:
#This code plots up afternoon-average footprints, but wihout the stamen tiling

#Image out name? Input file name?
out_name='199509020000_-111.707982_40.260787_0_foot.png'
foot_fname='/home/hhpower/Documents/TRI_STILT/data/processed/stilt_output/netcdf/199509020000_-111.707982_40.260787_0_foot.nc'
bath_fname='/home/hhpower/Documents/Plotting/GSL_bathymetry_new.nc'

#Read in our data foot print data
ncdf_foot=netcdf_dataset(foot_fname)
foot=ncdf_foot.variables['foot'][:,:]
flats=ncdf_foot.variables['lat'][:]
flons=ncdf_foot.variables['lon'][:]

#Since we can't use Stamen tiles, we must use an alternative data set for plotting "terrain". The bathemetry data set
ncdf_bath=netcdf_dataset(bath_fname)
terrain_h=ncdf_bath.variables['bathym'][:,:]
blats=ncdf_bath.variables['latitude'][:]
blons=ncdf_bath.variables['longitude'][:]

fig=plt.figure(figsize = (10, 10), dpi=150)
ax=plt.axes(projection=ccrs.PlateCarree())

#Set the extents of our domain and image resolution
ax.set_extent([-114, -110.25, 37.00, 41.50])

reader=Reader('/home/hhpower/Documents/Plotting/countyp010g/countyp010g.shp')
counties=list(reader.geometries())
COUNTIES=ShapelyFeature(counties, ccrs.PlateCarree())
ax.add_feature(COUNTIES, facecolor='none', edgecolor='black', linewidth=2)

#Lets create some plots
flevels=np.arange(-0.1,0.25,0.005)
tlevels=np.arange(1200,3200,200)

plot_foot=foot
plot_foot=gaussian_filter(plot_foot, 0.5)
plot_foot=np.reshape(plot_foot, (plot_foot.shape[0]*plot_foot.shape[1], plot_foot.shape[2]))
#plot_foot[plot_foot< -3.5]=no.nan

flon2d, flat2d = np.meshgrid(flons, flats)

##contour_terrain = plt.contourf(blons,blats,terrain_h,levels=tlevels,transform=ccrs.PlateCarree(),cmap=get_cmap("binary"),extend='both',alpha=1)
contour_foot = plt.contourf(flons,flats,plot_foot,levels=flevels,transform=ccrs.PlateCarree(),cmap=get_cmap("jet"),extend='max',alpha=0.5)

cbar=plt.colorbar(contour_foot, ax=ax, shrink=0.70, aspect=20)
cbar.set_label('Footprint [log(ppm/umoles/m2 s)]', rotation=270, weight='bold', size=18, labelpad=30)
cbar.ax.tick_params(labelsize=16)

#Save the output to a file
plt.savefig(out_name)
plt.close(fig)
os.system('mogrify -trim'+out_name+' '+out_name)

256

256

In [28]:

print(fig)


Figure(1500x1500)


In [83]:
print(ncdf_foot.variables)

{'lon': <class 'netCDF4._netCDF4.Variable'>
float64 lon(lon)
    units: degrees_east
    standard_name: longitude
    long_name: longitude at cell center
unlimited dimensions: 
current shape = (512,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'lat': <class 'netCDF4._netCDF4.Variable'>
float64 lat(lat)
    units: degrees_north
    standard_name: latitude
    long_name: latitude at cell center
unlimited dimensions: 
current shape = (512,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'time': <class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    units: seconds since 1970-01-01 00:00:00Z
    standard_name: time
    long_name: utc time
    calendar: standard
unlimited dimensions: 
current shape = (1,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'foot': <class 'netCDF4._netCDF4.Variable'>
float32 foot(time, lat, lon)
    units: ppm (umol-1 m2 s)
    _FillValue: -1.0
    standard_name: footprint
    long_name: stilt surface influence f

In [24]:
pf=np.reshape(plot_foot, (plot_foot.shape[0]*plot_foot.shape[1], plot_foot.shape[2]))

In [25]:
print(pf.shape)

(512, 512)
